In [3]:
import os

In [4]:
%pwd

'c:\\Users\\User\\Desktop\\Project Anchor\\MLflow-DVC-Kidney-Disease-Classification\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\User\\Desktop\\Project Anchor\\MLflow-DVC-Kidney-Disease-Classification'

In [7]:
# Create entity for prepare base model
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    """
    Configuration class for preparing the base model.

    Attributes:
        root_dir (Path): Root directory for storing artifacts.
        base_model_path (Path): File path to the base model.
        updated_base_model_path (Path): File path to the updated base model.
        params_image_size (list): Size of the input images [height, width, channels].
        params_learning_rate (float): Learning rate used in the optimizer.
        params_include_top (bool): Whether to include the top (fully connected) layers of the model.
        params_weights (str): Pre-trained weights to initialize the model.
        params_classes (int): Number of classes in the classification problem.
    """
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int


In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [9]:
# Class for managing configuration settings in prepare base model
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        """
        Initializes ConfigurationManager with configuration and parameter file paths.

        Args:
            config_filepath (str): Path to the configuration file.
            params_filepath (str): Path to the parameters file.
        """
        # Read configuration and parameters from files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create directories as per configuration
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        """
        Gets the configuration for preparing the base model.

        Returns:
            PrepareBaseModelConfig: Configuration for preparing the base model.
        """
        # Get configuration settings for preparing the base model
        config = self.config.prepare_base_model

        # Create directories specified in the configuration
        create_directories([config.root_dir])

        # Create PrepareBaseModelConfig object with obtained configuration settings
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config


In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [11]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        """
        Initializes PrepareBaseModel with the given configuration.

        Args:
            config (PrepareBaseModelConfig): Configuration for preparing the base model.
        """
        self.config = config

    def get_base_model(self):
        """
        Loads the base model and saves it.

        This method loads the base model, sets its configuration,
        and saves the model to the specified path.
        """
        # Load base model
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        # Save the base model
        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        """
        Prepares the full model by adding additional layers.

        Args:
            model (tf.keras.Model): Base model.
            classes (int): Number of classes.
            freeze_all (bool): Whether to freeze all layers.
            freeze_till (int): Number of layers to freeze.
            learning_rate (float): Learning rate for optimizer.

        Returns:
            tf.keras.Model: Prepared full model.
        """
        # Freeze layers as per configuration
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        # Add additional layers
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        # Compile the full model
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        # Print model summary
        full_model.summary()
        return full_model

    def update_base_model(self):
        """
        Updates the base model with additional layers and saves it.

        This method prepares the full model, updates the base model
        with additional layers, and saves the updated model to the specified path.
        """
        # Prepare the full model
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        # Save the updated base model
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """
        Saves the model to the specified path.

        Args:
            path (Path): Path to save the model.
            model (tf.keras.Model): Model to be saved.
        """
        model.save(path)


In [12]:
#update pipeline
try:
    # Initialize ConfigurationManager to get configuration
    config_manager = ConfigurationManager()

    # Get configuration for preparing the base model
    prepare_base_model_config = config_manager.get_prepare_base_model_config()

    # Initialize PrepareBaseModel with the obtained configuration
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)

    # Load and save the base model
    prepare_base_model.get_base_model()

    # Update and save the base model with additional layers
    prepare_base_model.update_base_model()

except Exception as e:
    # If any exception occurs, raise it and print error information
    raise e


[2024-03-05 21:16:18,178: INFO: common: YAML file 'config\config.yaml' loaded successfully]
[2024-03-05 21:16:18,180: INFO: common: YAML file 'params.yaml' loaded successfully]
[2024-03-05 21:16:18,180: INFO: common: Created directory at: artifacts]
[2024-03-05 21:16:18,180: INFO: common: Created directory at: artifacts/prepare_base_model]


[2024-03-05 21:16:19,363: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                          